In [27]:
puzzle_input = """####.
.##..
##.#.
###..
##..#"""

In [31]:
from collections import namedtuple

def part_1_solution(raw_input):
    grid = parse_grid(raw_input)
    
    sim = simulate(grid.height, grid.width)
    seen_grids = set()
    
    while grid not in seen_grids:
        seen_grids.add(grid)
        grid = process_one_minute(grid, sim)            
    
    return grid.biodiversity_rating()

BUG = '#'
EMPTY = '-'
    
class Grid:
    def __init__(self, height, width):
        self.height = height
        self.width = width
        self.rows = []
        for _ in range(self.height):
            self.rows.append([EMPTY] * self.width)

    def __str__(self):
        output = ''
        for row in self.rows:
            for cell in row:
                output += cell
            output += '\n'
        return output
    
    def __hash__(self):
        flattened = list()
        for row in self.rows:
            for obj in row:
                flattened.append(obj)
        
        return hash("".join(flattened))
    
    def __eq__(self, other):
        return self.rows == other.rows
    
    def biodiversity_rating(self):
        rating = 0
        i = 0
        for row in self.rows:
            for obj in row:
                if obj == BUG:
                    rating += 2**i
                i += 1
                
        return rating

    def query(self, y, x):
        if y >= self.height or x >= self.width:
            return EMPTY
        
        if y < 0 or x < 0:
            return EMPTY
        
        return self.rows[y][x]

    def assign(self, y, x, state):
        self.rows[y][x] = state


def parse_grid(raw_input):
    grid = Grid(5, 5)
    
    for y, row in enumerate(raw_input.split("\n")):
        for x, obj in enumerate(row):
            if obj == "#":
                grid.assign(y, x, BUG)
                
    return grid


def process_one_minute(grid, sim):
    new_grid = Grid(grid.height, grid.width)
    item = next(sim)
    while item is not TICK:
        if isinstance(item, Query):
            state = grid.query(item.y, item.x)
            item = sim.send(state)
        else:  
            new_grid.assign(item.y, item.x, item.state)
            item = next(sim)
    return new_grid
    
TICK = object()

def simulate(height, width):
    while True:
        for y in range(height):
            for x in range(width):
                yield from step_cell(y, x)
        yield TICK

        
Query = namedtuple('Query', ('y', 'x'))

def count_neighbors(y, x):
    n_ = yield Query(y + 1, x + 0)  
    e_ = yield Query(y + 0, x + 1)
    s_ = yield Query(y - 1, x + 0)
    w_ = yield Query(y + 0, x - 1)
    
    neighbor_states = [n_, e_, s_, w_]
    count = 0
    for state in neighbor_states:
        if state == BUG:
            count += 1
            
    return count

Transition = namedtuple('Transition', ('y', 'x', 'state'))

def step_cell(y, x):
    state = yield Query(y, x)
    neighbors = yield from count_neighbors(y, x)
    next_state = game_logic(state, neighbors)
    yield Transition(y, x, next_state)
    

def game_logic(state, neighbors):
    if state == BUG:
        if neighbors == 1:
            return BUG
        else:
            return EMPTY
    
    if neighbors == 1 or neighbors == 2:
        return BUG
    
    return EMPTY


In [32]:
test_input = """....#
#..#.
#..##
..#..
#...."""
assert(part_1_solution(test_input) == 2129920)
print("Test passed")

Test passed


In [33]:
print(f"Part 1 solution: {part_1_solution(puzzle_input)}")

Part 1 solution: 18400821


4.18 ms ± 304 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
